# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/Ironhack/Laboratorios/02 Módulo 2/lab-intro-recommender-systems/data/customer_product_sales.csv')

In [ ]:
#data = pd.read_csv('./data/customer_product_sales.csv')

In [5]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [7]:
# Dataframe with the total products purchased by customer
product_cust = data.groupby(['CustomerID', 'ProductName'], as_index = False).agg({'Quantity':'sum'}) # as_index=False avoids considering any given variable as an index 
product_cust.head(3)

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [9]:
# Matrix with all the products
piv_table = data.pivot_table(index = 'ProductName', columns = 'CustomerID', values = 'Quantity').fillna(0)
piv_table.head(3)

CustomerID,33,200,264,356,412,464,477,639,649,669,694,756,883,891,1008,1034,1066,1072,1336,1428,1435,1534,1577,1594,1754,1839,1920,2187,2329,2503,2556,2566,2582,2617,2686,2754,2776,2902,2915,2939,...,94438,94547,94599,94910,94951,95017,95034,95059,95078,95121,95314,95372,95819,96024,96088,96272,96522,96524,96560,96615,96666,96684,97029,97052,97063,97093,97201,97282,97324,97495,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,25.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,25.0,0.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [10]:
from scipy.spatial.distance import pdist,squareform

# scipy.spatial.distance.squareform(X, force='no', checks=True)
# Convert a vector-form distance vector to a square-form distance matrix, and vice-versa.

# Matrix with the distnaces between customer according to their product preferences
distances = pd.DataFrame(1/ (1 + squareform(pdist(piv_table.T, 'euclidean'))), index = piv_table.columns, columns = piv_table.columns) # piv_table.columns is CustomerID 
distances.head(3) 

CustomerID,33,200,264,356,412,464,477,639,649,669,694,756,883,891,1008,1034,1066,1072,1336,1428,1435,1534,1577,1594,1754,1839,1920,2187,2329,2503,2556,2566,2582,2617,2686,2754,2776,2902,2915,2939,...,94438,94547,94599,94910,94951,95017,95034,95059,95078,95121,95314,95372,95819,96024,96088,96272,96522,96524,96560,96615,96666,96684,97029,97052,97063,97093,97201,97282,97324,97495,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,0.087779,0.091747,0.086333,0.088913,0.088530,0.088530,0.089301,0.092176,0.087779,0.085297,0.084959,0.090499,0.092610,0.087410,0.092176,0.083972,0.092176,0.090499,0.085297,0.093498,0.089301,0.089695,0.088913,0.087779,0.087047,0.084626,0.089695,0.087047,0.090909,0.088530,...,0.004975,0.005548,0.005232,0.005284,0.004596,0.004993,0.005354,0.004712,0.005108,0.004970,0.004903,0.005128,0.004748,0.004740,0.004970,0.005039,0.005480,0.004871,0.004529,0.005115,0.005453,0.004803,0.005235,0.005086,0.005036,0.005358,0.005246,0.004754,0.004547,0.005249,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,0.083972,0.083972,0.083972,0.086333,0.085983,0.082709,0.088152,0.092610,0.081503,0.082403,0.082100,0.082403,0.084626,0.086333,0.086333,0.082403,0.094414,0.085638,0.084959,0.088152,0.085983,0.083020,0.086333,0.083972,0.081503,0.081800,0.086333,0.083972,0.085983,0.088913,...,0.004981,0.005564,0.005255,0.005291,0.004608,0.004996,0.005349,0.004722,0.005121,0.004978,0.004894,0.005131,0.004750,0.004748,0.004954,0.005032,0.005508,0.004897,0.004540,0.005127,0.005440,0.004816,0.005252,0.005069,0.005042,0.005338,0.005245,0.004766,0.004560,0.005259,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,0.088152,0.088913,0.090499,0.087779,0.092176,0.088152,0.089695,0.092610,0.083972,0.085638,0.083333,0.087779,0.088152,0.090094,0.088530,0.082403,0.092610,0.087047,0.083651,0.088913,0.088913,0.087779,0.090094,0.090499,0.080920,0.087047,0.089301,0.089695,0.093051,0.088913,...,0.004990,0.005548,0.005252,0.005280,0.004596,0.004999,0.005354,0.004715,0.005115,0.004961,0.004880,0.005118,0.004740,0.004743,0.004957,0.005036,0.005505,0.004880,0.004522,0.005108,0.005437,0.004811,0.005257,0.005070,0.005042,0.005350,0.005264,0.004740,0.004566,0.005238,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [19]:
# Selected customer 33

top5_cust33 = distances[33].sort_values(ascending= False)[1:6]
top5_cust33

CustomerID
3909    0.095358
3531    0.093953
264     0.093953
2503    0.093498
3305    0.093051
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [24]:
top5_cust33_friends = data[data['CustomerID'].isin(top5_cust33.index)].sort_values('CustomerID') # top_cust33.index
top5_cust33_friends.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
32311,264,Virgil,Small,1609342,240,Wine - Chablis 2003 Champs,1
34764,264,Virgil,Small,4356324,289,Sword Pick Asst,1
33544,264,Virgil,Small,2585657,361,Coffee Decaf Colombian,1
32037,264,Virgil,Small,6577594,297,Hinge W Undercut,1
29028,264,Virgil,Small,5538534,151,"Mushrooms - Black, Dried",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [30]:
top_products = top5_cust33_friends.groupby('ProductName').agg({'Quantity':'sum'}).sort_values('Quantity', ascending = False).reset_index()

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [32]:
# merge top_products with the matrix with all the products
merged_top5_friends = top_products.merge(piv_table, on = 'ProductName')

# From merged_top5_friendsselect the variables 'ProductName', 'Quantity', and '33'
total_products = merged_top5_friends[['ProductName', 'Quantity', 33]]

# From allo the products, select those ones customer 33 havent purchased yet
not_purshaed33 = total_products[total_products[33] == 0].reset_index(drop=True)

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [56]:
customers = data['CustomerID'].unique()
recommendations = {}

for customer in customers:
  top5_cust33 = distances[customer].sort_values(ascending= False)[1:6]
  top5_cust33_friends = data[data['CustomerID'].isin(top5_cust33.index)].sort_values('CustomerID')
  top_products = top5_cust33_friends.groupby('ProductName').agg({'Quantity':'sum'}).sort_values('Quantity', ascending = False).reset_index()
  merged_top5_friends = top_products.merge(piv_table, on = 'ProductName')
  total_products = merged_top5_friends[['ProductName', 'Quantity', customer]]
  not_purshaed33 = total_products[total_products[customer] == 0].reset_index(drop=True)
  recommendations[customer] = list(not_purshaed33['ProductName'][:5])
recommendations

{61288: ['Mushrooms - Black, Dried',
  'Wine - Magnotta, Merlot Sr Vqa',
  'Chicken - Soup Base',
  'Wine - Chardonnay South',
  'Milk - 1%'],
 77352: ['Guinea Fowl',
  'Grenadine',
  'Ecolab - Mikroklene 4/4 L',
  'Oranges - Navel, 72',
  'Coconut - Shredded, Sweet'],
 40094: ['Water - Mineral, Natural',
  'Pasta - Orecchiette',
  'Oregano - Dry, Rubbed',
  'Quiche Assorted',
  'Beef - Inside Round'],
 23548: ['Wanton Wrap',
  'Banana Turning',
  'Towels - Paper / Kraft',
  'Bread - Raisin Walnut Oval',
  'Lettuce - Treviso'],
 78981: ['Lettuce - Frisee',
  'Longos - Chicken Wings',
  'Pop Shoppe Cream Soda',
  'Sprouts - Alfalfa',
  'Beef - Inside Round'],
 83106: ['Cheese - Boursin, Garlic / Herbs',
  'Garlic - Peeled',
  'Soup - Campbells Tomato Ravioli',
  'Beef - Rib Eye Aaa',
  'Wine - Chablis 2003 Champs'],
 11253: ['Smirnoff Green Apple Twist',
  'Juice - Cranberry, 341 Ml',
  'Tomatoes Tear Drop',
  'Lettuce - Treviso',
  'Spinach - Baby'],
 35107: ['Flavouring - Orange',
  '

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [57]:
recommendation_df = pd.DataFrame.from_dict(recommendations).T.reset_index()
recommendation_df.columns =['CustomerID', 'Option_1', 'Option_2', 'Option_3', 'Option_4', 'Option_5']
recommendation_df.tail(3)

,CustomerID,Option_1,Option_2,Option_3,Option_4,Option_5
997,85878,Cookie - Dough Variety,"Beef - Ground, Extra Lean, Fresh",Muffin - Carrot Individual Wrap,Bread - Rye,Cheese - Camembert
998,68506,Pasta - Cheese / Spinach Bauletti,"Doilies - 5, Paper",Black Currants,Tea - Decaf Lipton,Cocoa Butter
999,3472,Muffin - Carrot Individual Wrap,"Cheese - Boursin, Garlic / Herbs",Pork - Kidney,Raspberries - Fresh,"Pasta - Detalini, White, Fresh"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [59]:
distance_cosine = pd.DataFrame(1/ (1 + squareform(pdist(piv_table.T, 'cosine'))), index = piv_table.columns, columns = piv_table.columns) # piv_table.columns is CustomerID 

In [61]:
customers= data['CustomerID'].unique()
recommendations_2 = {}

for customer in customers:
  top5_cust33 = distance_cosine[customer].sort_values(ascending= False)[1:6]
  top5_cust33_friends = data[data['CustomerID'].isin(top5_cust33.index)].sort_values('CustomerID')
  top_products = top5_cust33_friends.groupby('ProductName').agg({'Quantity':'sum'}).sort_values('Quantity', ascending = False).reset_index()
  merged_top5_friends = top_products.merge(piv_table, on = 'ProductName')
  total_products = merged_top5_friends[['ProductName', 'Quantity', customer]]
  not_purshaed33 = total_products[total_products[customer] == 0].reset_index(drop=True)
  recommendations_2[customer] = list(not_purshaed33['ProductName'][:5])
recommendations_2

recommendation_df_2 = pd.DataFrame.from_dict(recommendations_2).T.reset_index()
recommendation_df_2.columns =['CustomerID', 'Option_1', 'Option_2', 'Option_3', 'Option_4', 'Option_5']
recommendation_df_2.tail(3)

,CustomerID,Option_1,Option_2,Option_3,Option_4,Option_5
997,85878,Cheese - Camembert,Water - Green Tea Refresher,Halibut - Steaks,Bread - Rye,Apricots - Dried
998,68506,"Oranges - Navel, 72",Hot Chocolate - Individual,Beef - Ground Medium,Spinach - Baby,Muffin Chocolate Individual Wrap
999,3472,Flavouring - Orange,Hersey Shakes,Napkin White - Starched,Beans - Kidney White,Grenadine
